In [2]:
%load_ext autoreload
%autoreload 2

import os
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import concurrent.futures

import warnings
from typing import Self
from datetime import datetime
from functools import reduce
from sklearn.metrics import precision_recall_curve, auc, roc_curve
from concurrent.futures import ThreadPoolExecutor, wait, FIRST_COMPLETED

import src.utils.anomalydetectors as m
import src.utils.globals as g
import src.utils.aggregators as agg

from src.utils.experiment import Experiment
from src.utils.noise import NoiseFactory
from src.utils.plotting import plot_rpcurves

In [3]:
def name_experiment(prefix, ratio, intensity, robotids):
    name = f"{prefix}_{ratio}_{intensity}"

    if robotids:
        name = name + f"_{'_'.join(item.strip() for item in robotids)}"

    return name

def name_guassian_experiment(ratio, intensity, robotids):
    return name_experiment("gaussian", ratio, intensity, robotids)

def name_sinusoidal_experiment(ratio, intensity, robotids):
    return name_experiment("sinusoidal", ratio, intensity, robotids)

def name_point_experiment(ratio, intensity, robotids):
    return name_experiment("point", ratio, intensity, robotids)

In [4]:
df_in = pd.read_parquet(g.path_tracks)

In [5]:
# Boolean whether to run the experiments or load
load_experiments = True #Keep This True

# Running Experiment Parameters
models = [m.ZScore(), m.MZScore(), m.LOF(), m.IF()]
exp_columns = ['torqueactual']
syn_column = 'torqueactual'

guassian_params     = [0.10, 0.25, 0.40, 0.55]
guassian_column_length = len(guassian_params)

sinusoidal_params   = [0.25, 0.5, 1, 2]
sinusoidal_column_length = len(sinusoidal_params)

point_params        = [1, 5, 10, 100]
point_column_length = len(point_params)

ids = None #ids = ['6640-102140-1']
synratio = 0.1

# Loading Experiment Parameters


# Loading Experiments

In [6]:
def load_experiments(params, name_func, exp_dict):
    for intensity in params:
        name = name_func(synratio, intensity, ids)
        print(f"Loading Experiment {name}")
        exp_dict[name] = Experiment.unpickle(name)
    print("Experiments Loaded")

# Load or Run Gaussian Experiments
exps_gaussian = {}
load_experiments(guassian_params, name_guassian_experiment, exps_gaussian)

# Load or Run Sinusoidal Experiments
exps_sinusoidal = {}
load_experiments(sinusoidal_params, name_sinusoidal_experiment, exps_sinusoidal)

# Load or Run Point Experiments
exps_point = {}
load_experiments(point_params, name_point_experiment, exps_point)


Loading Experiment gaussian_0.1_0.1
Loading Experiment gaussian_0.1_0.25
Loading Experiment gaussian_0.1_0.4
Loading Experiment gaussian_0.1_0.55
Experiments Loaded
Loading Experiment sinusoidal_0.1_0.25
Loading Experiment sinusoidal_0.1_0.5
Loading Experiment sinusoidal_0.1_1
Loading Experiment sinusoidal_0.1_2
Experiments Loaded
Loading Experiment point_0.1_1
Loading Experiment point_0.1_5
Loading Experiment point_0.1_10
Loading Experiment point_0.1_100
Experiments Loaded


In [15]:
def plot_feature_scores(exp, df):
    pass

In [16]:
exps_point['point_0.1_1'].results['df']

,seqid,timeindex_bin,z,mz,lof_20,if
0,6640-101753-1|2021-03-01|10:37:33|original,0,0.515508,0.536583,1.077839,-0.632978
1,6640-101753-1|2021-03-01|10:37:33|original,1,0.273435,0.256780,1.051236,-0.632235
2,6640-101753-1|2021-03-01|10:37:33|original,2,0.186219,0.284368,1.004520,-0.633339
3,6640-101753-1|2021-03-01|10:37:33|original,3,0.425970,0.494517,0.979433,-0.631505
4,6640-101753-1|2021-03-01|10:37:33|original,4,0.681010,0.748828,1.061008,-0.633235
...,...,...,...,...,...,...
13764659,7600-100498-1|2021-12-08|05:11:33|point,8120,NaN,NaN,0.960766,-0.738178
13764660,7600-100498-1|2021-12-08|05:11:33|point,8121,NaN,NaN,0.987678,-0.748009
13764661,7600-100498-1|2021-12-08|05:11:33|point,8122,NaN,NaN,0.991026,-0.745462
13764662,7600-100498-1|2021-12-08|05:11:33|point,8123,NaN,NaN,1.024336,-0.747402
